In [1]:
import multiprocess as mp
import platform

# This is required for pymc parallel evaluation in notebooks 
# But has to be change while using a python script 
# Use this following instruction instead
# if __name__ == "__main__":
#     if platform.system() != "Windows":
#         mp.set_start_method('spawn')
    
    # rest of your code body here inside the if __name__
if platform.system() != "Windows":
    
    mp.set_start_method('forkserver')

In [2]:
import Calibrate as cal #Runing the calibration process and gathering results
from calibs_utilities import get_all_priors, get_targets, load_data
from models.models import model1, model2, bcm_seir_age_strat, bcm_sir #All the models we design for the test
from Calibrate import plot_comparison_bars

# Combining tagets and prior with our summer2 model in a BayesianCompartmentalModel (bcm_model_1)
from estival.model import BayesianCompartmentalModel
from estival.sampling.tools import likelihood_extras_for_idata
from estival.sampling.tools import likelihood_extras_for_samples


import pandas as pd
import numpy as np
# import plotly.express as px
import matplotlib.pyplot as plt
from typing import List

import pymc as pm

# We use estivals parallel tools to run the model evaluations
from estival.utils.parallel import map_parallel

import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
import pickle
from datetime import datetime
# import jax
from jax import numpy as jnp
pd.options.plotting.backend = "plotly" #To allow plotly graphic. Swich to "matplotlib" if facing some troubles while ploting
# pd.options.plotting.backend = "matplotlib"



WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [6]:
model_config = {"compartments": ("S", "E","I","R"), # "Ip","Ic", "Is", "R"),
        "population": 56490045, #England population size 2021
        "seed": 50.0,
        "start": datetime(2020, 8, 1),
        "end_time": datetime(2020, 11, 14)
}



In [3]:
from pathlib import Path
from calibs_utilities import get_all_priors, get_targets
import estival.targets as est
import estival.priors as esp

In [12]:
#------DATA-MANAGEMENT------------------------"
df = pd.read_csv("./data/new_cases_England_2020.csv")
df["date"] = pd.to_datetime(df.date)
df.set_index(["age","date"], inplace=True)

# ages_labels = [f"{i:02}_{i+4:02}" for i in range(0,60, 5)] + ["60+"]
# targets_data = dict()
# for age in ages_labels:
#     targets_data[age] = df.loc[age]

ages_labels = [f"{i:02}_{i+4:02}" for i in range(0,60, 5)] + ["60+"]
age_strat = [f"{i}" for i in range(0,65,5)]

#Default parameters
parameters = {
'age_transmission_rate_'+ str(age) : 0.25 for age in age_strat
}
parameters['incubation_period']= 6
parameters['infectious_period'] = 7.3


#The fitted period is Aug 2020 to Nov 2020
# We define a normal target with fixed std for each age catergory
# We rolle the data for 14 day to discard fluctuations
# esp.UniformPrior("IXage_"+str(age)+"_disp",(0.1, 2000))
targets = [
    est.TruncatedNormalTarget("IXage_"+str(age), df.loc[age_cat]["Aug 2020":"Nov 19 2020"].rolling(14).mean().iloc[14:]["cases"],
                            (0.0,np.inf),
                    esp.UniformPrior("IXage_"+str(age)+"_disp",(0.1, 2000))) for age_cat, age in zip(ages_labels, age_strat)
]
# A uniform prior is defined for all the transmission rate
params = {param: (0.0,1.0) for param in (parameters.keys())}
priors = []
# A normal prior for the incubation and infectious periods
normal_priors = [ 
esp.TruncNormalPrior("incubation_period",5.4, 3.0, (1,15)),
esp.TruncNormalPrior("infectious_period",7.3, 2.0, (1,15)),
]
uniform_priors = get_all_priors(params)
priors = normal_priors + uniform_priors[:-2]

model_2 = model2(model_config)
#Defining  a Bayesian Compartmental Model

bcm_model_2 = BayesianCompartmentalModel(model_2, parameters, priors,targets, extra_ll=False)

Cannot acces the likelihood for the bcm while tempting to calibrate the targets dispersion.

Therefore, imposible to use NUTS from numpyro to call the likelihood

In [ ]:
bcm_model_2.run(parameters)

In [18]:
disp_params = {k:v.ppf(0.5) for k,v in bcm_model_2.priors.items() if "_disp" in k}


In [19]:
disp_params

{'IXage_0_disp': 1000.0500000000001,
 'IXage_5_disp': 1000.0500000000001,
 'IXage_10_disp': 1000.0500000000001,
 'IXage_15_disp': 1000.0500000000001,
 'IXage_20_disp': 1000.0500000000001,
 'IXage_25_disp': 1000.0500000000001,
 'IXage_30_disp': 1000.0500000000001,
 'IXage_35_disp': 1000.0500000000001,
 'IXage_40_disp': 1000.0500000000001,
 'IXage_45_disp': 1000.0500000000001,
 'IXage_50_disp': 1000.0500000000001,
 'IXage_55_disp': 1000.0500000000001,
 'IXage_60_disp': 1000.0500000000001}

In [20]:
def nmodel_2():
    unif_priors = list(bcm_model_2.parameters)[:-2]
    sampled = {k:numpyro.sample(k, dist.Uniform(0.0,1.0)) for k in unif_priors}
    #Adding the normal priors for the incubation and infectious periods
    sampled["incubation_period"] = numpyro.sample("incubation_period", dist.TruncatedNormal(7.3, 2.0, low=1., high=14.))
    sampled["infectious_period"] = numpyro.sample("infectious_period", dist.TruncatedNormal(5.4, 3.0, low=1., high=14.))
    # Log-likelihood
    disp_params = {k:v.ppf(0.5) for k,v in bcm_model_2.priors.items() if "_disp" in k}

    log_likelihood = bcm_model_2.loglikelihood(**sampled |disp_params)

    numpyro.factor("ll",log_likelihood)

In [21]:
from jax import random
from numpyro.infer import init_to_feasible, init_to_mean, init_to_sample, init_to_median,init_to_value, init_to_uniform

In [22]:
chains = 4
kernel = infer.NUTS(nmodel_2, target_accept_prob=0.75, dense_mass=True, init_strategy=init_to_sample())
mcmc = infer.MCMC(kernel, num_warmup=10, num_chains=chains, num_samples=100, progress_bar=True)

mcmc.run(random.PRNGKey(0))
idata1 = az.from_numpyro(mcmc)

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]